# Monte Carlo Tree Search Implementation

In [8]:
class Game:
    def __init__(self, board=0, player=1, winner=None):
        self.board = board
        self.player = player
        self.winner = winner
    def get_legal_moves(self):
        return [1, 2] if self.player == 1 else [-0.5, -1.5]
    
    def make_move(self, move):
        self.board += move
        self.player = -self.player
        self.winner = self.check_winner()

    def check_winner(self):
        if self.board >= 10:
            return 1
        elif self.board <= -10:
            return -1
        else:
            return 0
    
    def is_terminal(self):
        return self.winner is not None

    def get_reward(self):
        return self.winner
    
    def reset(self):
        self.board = 0
        self.player = 1
        self.winner = None

    def copy_state(self):
        return (self.board, self.player, self.winner)

    def render(self):
        print(f"Board: {self.board}, Player: {self.player}, Winner: {self.winner}")




In [13]:
import math
import random
C = math.sqrt(2)
class Node:
    def __init__(self, parent, move, game_copy):
        self.game = game_copy
        self.parent = parent
        self.move = move
        self.children = []
        self.untried_moves = self.game.get_legal_moves()
        self.visits = 0
        self.wins = 0
    def get_uct(self):
        if self.visits == 0:
            return float('inf')
        return self.wins / self.visits + C * math.sqrt(math.log(self.parent.visits) / self.visits)


    def selection(self):
        current = self
        while not current.game.is_terminal():
            if current.untried_moves:  # If there are untried moves, stop here
                return current
            if not current.children:   # If no children, stop here
                return current
            current = current.select_child()
        return current

        
    def select_child(self):
        return max(self.children, key=lambda child: child.get_uct())

    def expand(self):
        move = self.untried_moves.pop()
        game_copy = Game(*self.game.copy_state())
        game_copy.make_move(move)
        child = Node(self, move, game_copy)
        self.children.append(child)
        return child

    def rollout(self):
        game_copy = Game(*self.game.copy_state())
        while not game_copy.is_terminal():
            game_copy.make_move(random.choice(game_copy.get_legal_moves()))
        return game_copy.get_reward()
    
    def backpropagate(self, result):
        self.visits += 1
        self.wins += result
        if self.parent:
            self.parent.backpropagate(result * -1)


def mcts(game, iterations):
    root = Node(None, None, game)
    for _ in range(iterations):
        node = root.selection()
        if node.untried_moves:  # Add this check
            node = node.expand()
        result = node.rollout()
        node.backpropagate(result)

    return root.select_child().move

game = Game()
mcts(game, 100)

2